## Sochastic simulation algorithm (SSA)
Jens Hahn  -  06/06/2016


Last time we have talked about ODE modelling, which is a **deterministic** and **continuous** way of modelling. This time, we'll talk about a completely different approach, a **stochastic** and **discrete** simulation. Firstly, this means we get a slightly different result, everytime we run the simulation. Secondly, this means we talk about single molecules and not a vague float value that needs to be interpreted to a molecule count. 

### Gillespie's algorithm
The idea of Gillespie's algorithm is to take into account every reaction event, based on molecule collisions and reaction radii around the molecules. Based on all possible combinations of collisions 

We start with a specific parameter, called $c$, the average probability that a reactant molecule will react per unit time. The next parameter we need is called $h$, the number of distinct molecular reactant combinations for a given reaction at time t. Finally, we need the distinct time interval we're focus on, we call it $\delta t$.

In [4]:
import math
import numpy as np
a=1
r=1
t = -(1/a)*math.log(r)

In [14]:
np.random.uniform()

0.9586454093903716

In [ ]:
r = uniformRandomNumber()
Reactions index from 1 to numberOfReactions
Propensities index form 1 to numberOfReactions

if r <= n(1) then
    Reaction[1] fires
else
    begin
    for i = 2 to numberOfReactions do
        if (r > n[i-1]) and (r <= n[i]) then
            begin 
            Reaction[i] fires
            exit
            end
    end

In [ ]:
accumulate = 0
i = 0
scaledRandomNumber = uniformRandomNumber() * R
while accumulate < scaledRandomNumber do
    begin
    i = i + 1
    accumulate = accumulate + n[i]
    end;
Fire the ith reaction